## Make a DataFrame consolidating snaq results on the cluster.

### Each snaq run was trying to detect introgression from 1000 loci of 300bp apiece.

In [1]:
import numpy as np
import toytree
import os
import re
import toyplot
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

import statsmodels.api as sm
from statsmodels.formula.api import logit

### Lots of snaq runs finshed in separate jobs on the cluster. Here's a scenario with a balanced, 10-tip species tree that is 2e6 generations tall.

In [2]:
len(os.listdir("../bal_10k_2mil/may2020runs/"))

1000

In [3]:
os.listdir("../bal_10k_2mil/may2020runs/")[:10]

['705', '338', '630', '263', '334', '747', '69', '672', '743', '376']

### In each directory, there are snaq results and a metadata file with information about the simulated scenario.

Let's look at the metadata file for the first run:

In [4]:
with open("../bal_10k_2mil/may2020runs/1/meta_data.txt") as f:
    print(f.read())


newick=(((r6:995183,r5:995183)15:780017,(r7:1.22692e+06,(r9:542302,r8:542302)11:684614)14:548284)17:770606,((r1:1.23064e+06,r0:1.23064e+06)13:671941,(r2:1.42251e+06,(r4:564096,r3:564096)10:858418)12:480063)16:643229);
root_height=2545806.0207428304
popsizes=[600904.82211533 945078.42556007 203872.76377977 719233.59877209
 248695.98196572 636929.1519803  699703.88114598 560834.73207103
 162933.00179411 886479.53112118 142603.81060688 426131.06857097
 994216.73974058 655912.10955362 433439.13679275 692039.24499592
 652694.70227892 903087.9836565  862426.99855271]
source=8
dest=7
time=0.48795282276820184
magnitude=0.4800977604607741
recomb=5e-08
nloci=1000
nbp=300
seed=739986152



### Load in all of the snaq results:

In [5]:
# save whether snaq got the source/dest correct
correct_snaq = []
# save the magnitude of introgression from the simulation
magnitudes = []
# save the true source from the simulation
sources = []
# save the true dest from the simulation
dests = []

pseudo_liks_net1 = []
pseudo_liks_net0 = []

for loop in range(1000):
    working_dir = "../bal_10k_2mil/may2020runs/" + str(loop+1) + "/"
    
    # if the simulation finished... (many didn't)
    if "bestnet_h1.tre" in os.listdir(working_dir):
        with open(working_dir + "net1.out") as f:   #"bestnet_h1.tre") as f:
            # read in the snaq result
            #hybrid_new=f.read().split()[0]
            first = f.read().split('\n')[0].split()
            hybrid_new = first[0]
            pseudolik_net1 = first[-1]
        with open(working_dir + "net0.out") as f:   #"bestnet_h1.tre") as f:
            # read in the snaq result
            #hybrid_new=f.read().split()[0]
            first = f.read().split('\n')[0].split()
            #hybrid_new = first[0]
            pseudolik_net0 = first[-1]
        
        with open(working_dir + "meta_data.txt") as f:
            # read in the simulation metadata
            meta = f.read().split()
            
        # get the simulated newick tree (with node sliding and rood sliding)
        simnewick = meta[0].split("newick=")[1]
        # toytree-ify the newick
        simtopo = toytree.tree(simnewick)
        # get the simulated introgression magnitude
        magnitude = float(meta[np.argmax([i[:9]=='magnitude' for i in meta])].split('magnitude=')[1])
        # get the simulated source
        simsource = int(meta[np.argmax([i[:6]=='source' for i in meta])].split('source=')[1])
        # get the simulated dest
        simdest = int(meta[np.argmax([i[:4]=='dest' for i in meta])].split('dest=')[1])
        
        # we want to record the leaf names from below the introgression event to make the edges
        # on the toytree comparable to the snaq tree
        for i in simtopo.treenode.traverse():
            if i.idx == simsource:
                sim_source_descendant_leaves = [q.name for q in i.get_leaves()]
            if i.idx == simdest:
                sim_dest_descendant_leaves = [q.name for q in i.get_leaves()]
                
        # now parse the snaq output network tree
        # first, gotta remove the pound-signed hybrid edge
        # to do this, we have to break apart the newick by parantheses and commas
        fs = hybrid_new.split('(')
        ss = [i.split(')') for i in fs]
        ts = [[i.split(',') for i in q] for q in ss]
        activate=0
        for i in range(len(ts)):
            for q in range(len(ts[i])):
                for p in range(len(ts[i][q])):
                    if "#" in ts[i][q][p]:
                        #ts[i][q].pop(p)
                        ts[i][q][p] = ''
                        activate += 1
                    if activate:
                        break
                if activate:
                    break
            if activate:
                break

        activate=0
        for i in range(len(ts)):
            for q in range(len(ts[i])):
                for p in range(len(ts[i][q])):
                    if "#" in ts[i][q][p]:
                        #ts[i][q].pop(p)
                        ts[i][q][p] = ''
                        activate += 1
                    if activate:
                        break
                if activate:
                    break
            if activate:
                break
        
        # now, re-merge the newick file
        ss_new = [[",".join(p) for p in q] for q in ts]
        edge_removed_newick = "(".join([")".join(i) for i in ss_new])
        
        # remove commas that come before parantheses...
        edge_removed_newick = re.sub(",\)",")",edge_removed_newick)
        
        # rarely, the topology is misinferred, and this isn't a monophyletic group.
        try:
            ed_rem_tre = toytree.tree(edge_removed_newick).root(["r0","r1","r2","r3","r4"])
        except:
            pass
        
        # save the descendant leaves from the ends of the hybrid edge, to compare to the simulated results
        snaq_hybrid_descendant_leaves = []
        for currnode in ed_rem_tre.treenode.traverse():    
            if len(currnode.get_children()) == 1 and not currnode.is_leaf():
                snaq_hybrid_descendant_leaves.append([i.name for i in currnode.get_leaves()])
        sim_sourcedest = [sim_dest_descendant_leaves,sim_source_descendant_leaves]
        
        # do the snaq results match the simulated results? Just looking at the same two branches. 
        correct_snaq.append(sorted(["".join(sorted(i)) for i in snaq_hybrid_descendant_leaves]) == sorted(["".join(sorted(i)) for i in sim_sourcedest]))
        magnitudes.append(magnitude)
        sources.append(simsource)
        dests.append(simdest)
        pseudo_liks_net0.append(pseudolik_net0)
        pseudo_liks_net1.append(pseudolik_net1)

### Record which introgressive events are between sister edges...

In [6]:
sisters = []
for i in simtopo.treenode.traverse():
    if len(i.get_children()) == 2:
        sisters.append(sorted([q.idx for q in i.get_children()]))

In [7]:
sisters

[[16, 17],
 [14, 15],
 [12, 13],
 [8, 9],
 [7, 11],
 [3, 4],
 [2, 10],
 [5, 6],
 [0, 1]]

In [8]:
idx=0
which_sister = []
for i in zip(sources,dests):
    if sorted(i) in sisters:
        which_sister.append(idx)
        
    idx+=1

### now build the results table!

In [39]:
snaq_results_table = pd.DataFrame([sources,dests,magnitudes],index=['source','dest','magnitude']).T
snaq_results_table['sisters'] = False
snaq_results_table.sisters[which_sister] = True
snaq_results_table['pseudolik_0'] = pseudo_liks_net0
snaq_results_table['pseudolik_1'] = pseudo_liks_net1
snaq_results_table['correct_inf'] = correct_snaq

/rigel/home/pfm2119/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
snaq_results_table

,source,dest,magnitude,sisters,pseudolik_0,pseudolik_1,correct_inf
0,8.0,7.0,0.480098,False,256.8971302469049,28.469852266426845,True
1,14.0,2.0,0.337213,False,77.97219005918686,28.85545220031675,False
2,3.0,14.0,0.347723,False,386.0424276257895,46.64859602813979,False
3,17.0,16.0,0.077040,True,20.61844240345986,15.044714557780221,False
4,15.0,14.0,0.147675,True,21.061964883790758,14.912283411239912,False
...,...,...,...,...,...,...,...
845,14.0,16.0,0.365770,False,22.47259892182806,16.386109647669407,False
846,5.0,8.0,0.374643,False,613.2111363884136,33.33656874345728,True
847,3.0,5.0,0.441719,False,600.7316796139028,25.75708561329744,False
848,11.0,9.0,0.365026,False,183.02898105542369,34.15571925047308,False


In [42]:
snaq_results_table.to_csv("../data/snaq_results/bal_10tip_2mil/snaq_results.csv",index=False)